In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
water_year = 'wy2021'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

# Clouds

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Cover: 0 -> Full; 1 -> Clear'

In [ ]:
HRRR_cloud = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/cloud_factor.nc',
    parallel=True, chunks={'time': 24, 'y' :10, 'x': 10}
)

butte_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_cloud.TCDC.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()

del HRRR_cloud

In [ ]:
smrf_out = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw/run*/smrf_20*.nc',
    parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure']
)

In [ ]:
butte_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
butte_snobal = butte_snobal.squeeze(['x', 'y']).compute()
schofield_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
schofield_snobal = schofield_snobal.squeeze(['x', 'y']).compute()
taylor_snobal = smrf_out.cloud_factor.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
taylor_snobal = taylor_snobal.squeeze(['x', 'y']).compute()

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * schofield_snobal.hvplot(label='SMRF clouds') 

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * butte_snobal.hvplot(label='SMRF clouds') 

### Taylor 

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR clouds').opts(**HV_PLOT_OPTS) * taylor_snobal.hvplot(label='SMRF clouds') 

### Rolling (daily) 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte') * \
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor') + \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte')  * \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor')).cols(1)